In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
from dateutil import parser
from datetime import date, timedelta
import pytz
from pytz import timezone
from io import StringIO
from decimal import Decimal


In [2]:
no = ['03-30','03-36','04-02-a','04-02','04-05','04-06','04-11','04-17','04-21','04-27','05-01',\
'05-02-a','05-02-b','05-02','05-06','05-10','05-14']
PIR_whole = pd.DataFrame()
for no_ in no:
    path = "/Users/Susie/Desktop/Data From 1st April/"+no_+".csv"
    PIR_one = pd.read_csv(path)    
    PIR_whole = PIR_whole.append(PIR_one)

PIR_whole.head()    

,EventEnqueuedUtcTime,EventProcessedUtcTime,IoTHub.ConnectionDeviceGenerationId,IoTHub.ConnectionDeviceId,IoTHub.CorrelationId,IoTHub.EnqueuedTime,IoTHub.MessageId,IoTHub.StreamId,PartitionId,Unnamed: 0,address,name,taskLocation,taskName,timestamp,value
0,2017-03-30T00:12:44.6280000Z,2017-03-30T00:12:45.5116207Z,636147800058290124,SG-04-demo0003,NaN,0001-01-01T00:00:00.0000000,NaN,NaN,2,1,01:84:D6:97:00:03,movement,Bathroom,movement,2017-03-30T00:12:35.0000000Z,true
1,2017-03-30T00:12:47.1400000Z,2017-03-30T00:12:47.9647560Z,636147896218147947,SG-04-avent002,NaN,0001-01-01T00:00:00.0000000,NaN,NaN,0,2,01:84:D8:4E:00:1D,movement,Bathroom,movement,2017-03-30T00:12:36.0000000Z,true
2,2017-03-30T00:12:49.4870000Z,2017-03-30T00:12:50.4062013Z,636244570774155260,SG-04-testingK,NaN,0001-01-01T00:00:00.0000000,NaN,NaN,5,3,oic:ZWAVE_30_XXXXXXX,power_volt,Living Room,powerMeter,2017-03-30T00:12:34.0000000Z,24339
3,2017-03-30T00:12:50.8620000Z,2017-03-30T00:12:51.7364137Z,636147800058290124,SG-04-demo0003,NaN,0001-01-01T00:00:00.0000000,NaN,NaN,2,4,NaN,heartbeat,NaN,NaN,NaN,2017-03-30T00:12:50.0000000Z
4,2017-03-30T00:12:53.0610000Z,2017-03-30T00:12:53.9403998Z,636147895634468152,SG-04-avent001,NaN,0001-01-01T00:00:00.0000000,NaN,NaN,1,5,NaN,heartbeat,NaN,NaN,NaN,2017-03-30T00:12:51.0000000Z


In [3]:
outtime = PIR_whole[(PIR_whole['taskName']=='alert_button')&(PIR_whole['name']=='proximity')]

In [4]:
def setflag(timestamp,day,tz):
    flag = date.today()- timedelta(day)
    flag = flag.strftime('%Y-%m-%d')
    flag = flag + ' '+ timestamp
    flag = datetime.strptime(flag, "%Y-%m-%d %H:%M:%S")
    if tz!='normal':
        flag = flag.replace(tzinfo=pytz.timezone('Asia/Singapore'))
    else:
        flag = flag
    return flag


def time_normer(rawdata):
    rawdata = rawdata.dropna(subset=['EventProcessedUtcTime'],how='all')
    rawdata['time'] = rawdata['EventProcessedUtcTime'].apply(lambda x: str(x))
    rawdata['time'] = rawdata['time'].apply(lambda x: x[:19])
    rawdata['time'] = rawdata['time'].apply(lambda x: datetime.strptime(x,'%Y-%m-%dT%H:%M:%S')+timedelta(hours=8))
    rawdata['eventtime'] = rawdata['time'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')) 
    return rawdata


In [5]:
PIR_whole.columns.values

array(['EventEnqueuedUtcTime', 'EventProcessedUtcTime',
       'IoTHub.ConnectionDeviceGenerationId', 'IoTHub.ConnectionDeviceId',
       'IoTHub.CorrelationId', 'IoTHub.EnqueuedTime', 'IoTHub.MessageId',
       'IoTHub.StreamId', 'PartitionId', 'Unnamed: 0', 'address', 'name',
       'taskLocation', 'taskName', 'timestamp', 'value'], dtype=object)

In [6]:
PIR_whole = time_normer(PIR_whole)
flag = setflag("00:00:00",60,'normal')
PIR_whole = PIR_whole[PIR_whole['name']=='movement']

In [7]:
PIR_whole = PIR_whole[PIR_whole['IoTHub.ConnectionDeviceId']=='SG-04-avent001']

In [8]:
PIR_whole = PIR_whole.sort('EventProcessedUtcTime', ascending=1)

/Users/Susie/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [9]:
PIR_whole.index = range(PIR_whole.shape[0])

In [10]:
PIR_whole['taskLocation'].unique()

array(['Bedroom', 'Bathroom', 'Kitchen', 'Living Room', 'Store Room',
       'Entrance'], dtype=object)

In [11]:
PIR_whole.head()

,EventEnqueuedUtcTime,EventProcessedUtcTime,IoTHub.ConnectionDeviceGenerationId,IoTHub.ConnectionDeviceId,IoTHub.CorrelationId,IoTHub.EnqueuedTime,IoTHub.MessageId,IoTHub.StreamId,PartitionId,Unnamed: 0,address,name,taskLocation,taskName,timestamp,value,time,eventtime
0,2017-03-26T18:31:54.3070000Z,2017-03-26T18:31:55.4909507Z,636147895634468152,SG-04-avent001,NaN,0001-01-01T00:00:00.0000000,NaN,NaN,1,536,01:84:D8:90:00:08,movement,Bedroom,movement,2017-03-26T18:31:43.0000000Z,true,2017-03-27 02:31:55,2017-03-27 02:31:55
1,2017-03-26T18:32:01.5600000Z,2017-03-26T18:32:08.9165613Z,636147895634468152,SG-04-avent001,NaN,0001-01-01T00:00:00.0000000,NaN,NaN,1,538,01:84:D8:90:00:08,movement,Bedroom,movement,2017-03-26T18:31:52.0000000Z,true,2017-03-27 02:32:08,2017-03-27 02:32:08
2,2017-03-26T18:32:09.7830000Z,2017-03-26T18:32:17.5574579Z,636147895634468152,SG-04-avent001,NaN,0001-01-01T00:00:00.0000000,NaN,NaN,1,539,01:84:D8:90:00:08,movement,Bedroom,movement,2017-03-26T18:32:01.0000000Z,true,2017-03-27 02:32:17,2017-03-27 02:32:17
3,2017-03-26T18:32:18.9750000Z,2017-03-26T18:32:20.0745473Z,636147895634468152,SG-04-avent001,NaN,0001-01-01T00:00:00.0000000,NaN,NaN,1,541,01:84:D8:90:00:0A,movement,Bathroom,movement,2017-03-26T18:32:09.0000000Z,true,2017-03-27 02:32:20,2017-03-27 02:32:20
4,2017-03-26T18:32:35.3420000Z,2017-03-26T18:32:36.4314697Z,636147895634468152,SG-04-avent001,NaN,0001-01-01T00:00:00.0000000,NaN,NaN,1,544,01:84:D8:90:00:0A,movement,Bathroom,movement,2017-03-26T18:32:26.0000000Z,true,2017-03-27 02:32:36,2017-03-27 02:32:36


In [12]:
PIR_whole.shape

(85020, 18)

In [13]:
def get_grouped(rawdata,room):
    rawdata = rawdata[rawdata['taskLocation']==room]
    rawdata = rawdata.set_index(rawdata['eventtime'].map(parser.parse))
    if rawdata.shape[0]!=0:
        grouped = rawdata.groupby(pd.TimeGrouper('60s')).size()
    return grouped
def timetable_generator(room,name):
    room['time'] = room.index
    room = room.rename(index=str, columns={ 0: name})
    room.index = range(room.shape[0])
    return room

In [14]:
outtime = outtime[outtime['IoTHub.ConnectionDeviceId']=='SG-04-avent001']
outtime = time_normer(outtime)
outtime.head()

,EventEnqueuedUtcTime,EventProcessedUtcTime,IoTHub.ConnectionDeviceGenerationId,IoTHub.ConnectionDeviceId,IoTHub.CorrelationId,IoTHub.EnqueuedTime,IoTHub.MessageId,IoTHub.StreamId,PartitionId,Unnamed: 0,address,name,taskLocation,taskName,timestamp,value,time,eventtime
687,2017-03-30T00:51:33.5970000Z,2017-03-30T00:51:34.3947000Z,636147895634468152,SG-04-avent001,NaN,0001-01-01T00:00:00.0000000,NaN,NaN,1,688,D0:5F:B8:32:4C:04,proximity,Landing,alert_button,2017-03-30T00:51:28.0000000Z,false,2017-03-30 08:51:34,2017-03-30 08:51:34
2366,2017-03-30T03:14:02.8680000Z,2017-03-30T03:14:03.7026582Z,636147895634468152,SG-04-avent001,NaN,0001-01-01T00:00:00.0000000,NaN,NaN,1,2367,D0:5F:B8:32:4C:04,proximity,Landing,alert_button,2017-03-30T03:13:57.0000000Z,true,2017-03-30 11:14:03,2017-03-30 11:14:03
4656,2017-03-30T05:10:15.0710000Z,2017-03-30T05:10:15.9301510Z,636147895634468152,SG-04-avent001,NaN,0001-01-01T00:00:00.0000000,NaN,NaN,1,4657,D0:5F:B8:32:4C:04,proximity,Landing,alert_button,2017-03-30T05:10:10.0000000Z,false,2017-03-30 13:10:15,2017-03-30 13:10:15
4694,2017-03-30T05:12:40.3410000Z,2017-03-30T05:12:41.1774330Z,636147895634468152,SG-04-avent001,NaN,0001-01-01T00:00:00.0000000,NaN,NaN,1,4695,D0:5F:B8:32:4C:04,proximity,Landing,alert_button,2017-03-30T05:12:35.0000000Z,true,2017-03-30 13:12:41,2017-03-30 13:12:41
5966,2017-03-30T06:27:52.7990000Z,2017-03-30T06:27:53.7976189Z,636147895634468152,SG-04-avent001,NaN,0001-01-01T00:00:00.0000000,NaN,NaN,1,5967,D0:5F:B8:32:4C:04,proximity,Landing,alert_button,2017-03-30T06:27:47.0000000Z,false,2017-03-30 14:27:53,2017-03-30 14:27:53


# get time table

In [15]:
bathroom = get_grouped(PIR_whole,'Bathroom').to_frame()
bathroom = timetable_generator(bathroom,'bathroom')
kitchen = get_grouped(PIR_whole,'Kitchen').to_frame()
kitchen = timetable_generator(kitchen,'kitchen')
livingroom = get_grouped(PIR_whole,'Living Room').to_frame()
livingroom = timetable_generator(livingroom,'livingroom')

## deal with outdoor data

In [16]:
outtime['eventtime'] = outtime['eventtime'].apply(lambda x: x[:16]) 
outtime.head()

,EventEnqueuedUtcTime,EventProcessedUtcTime,IoTHub.ConnectionDeviceGenerationId,IoTHub.ConnectionDeviceId,IoTHub.CorrelationId,IoTHub.EnqueuedTime,IoTHub.MessageId,IoTHub.StreamId,PartitionId,Unnamed: 0,address,name,taskLocation,taskName,timestamp,value,time,eventtime
687,2017-03-30T00:51:33.5970000Z,2017-03-30T00:51:34.3947000Z,636147895634468152,SG-04-avent001,NaN,0001-01-01T00:00:00.0000000,NaN,NaN,1,688,D0:5F:B8:32:4C:04,proximity,Landing,alert_button,2017-03-30T00:51:28.0000000Z,false,2017-03-30 08:51:34,2017-03-30 08:51
2366,2017-03-30T03:14:02.8680000Z,2017-03-30T03:14:03.7026582Z,636147895634468152,SG-04-avent001,NaN,0001-01-01T00:00:00.0000000,NaN,NaN,1,2367,D0:5F:B8:32:4C:04,proximity,Landing,alert_button,2017-03-30T03:13:57.0000000Z,true,2017-03-30 11:14:03,2017-03-30 11:14
4656,2017-03-30T05:10:15.0710000Z,2017-03-30T05:10:15.9301510Z,636147895634468152,SG-04-avent001,NaN,0001-01-01T00:00:00.0000000,NaN,NaN,1,4657,D0:5F:B8:32:4C:04,proximity,Landing,alert_button,2017-03-30T05:10:10.0000000Z,false,2017-03-30 13:10:15,2017-03-30 13:10
4694,2017-03-30T05:12:40.3410000Z,2017-03-30T05:12:41.1774330Z,636147895634468152,SG-04-avent001,NaN,0001-01-01T00:00:00.0000000,NaN,NaN,1,4695,D0:5F:B8:32:4C:04,proximity,Landing,alert_button,2017-03-30T05:12:35.0000000Z,true,2017-03-30 13:12:41,2017-03-30 13:12
5966,2017-03-30T06:27:52.7990000Z,2017-03-30T06:27:53.7976189Z,636147895634468152,SG-04-avent001,NaN,0001-01-01T00:00:00.0000000,NaN,NaN,1,5967,D0:5F:B8:32:4C:04,proximity,Landing,alert_button,2017-03-30T06:27:47.0000000Z,false,2017-03-30 14:27:53,2017-03-30 14:27


In [17]:
outtime['eventtime'] = outtime['eventtime'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M'))
outtime = outtime[['eventtime','value']]

In [18]:
outtime = outtime.rename(index=str, columns={ 'eventtime': 'time'})
outtime = outtime.rename(index=str, columns={ 'value': 'outdoor'})

In [19]:
outtime.head()

,time,outdoor
687,2017-03-30 08:51:00,false
2366,2017-03-30 11:14:00,true
4656,2017-03-30 13:10:00,false
4694,2017-03-30 13:12:00,true
5966,2017-03-30 14:27:00,false


## deal with RMS data

In [20]:
rms = pd.read_csv("/Users/Susie/Desktop/RMS_advent01_from_1st_April/Advent01.csv")

In [21]:
def time_normer(rawdata):
    rawdata = rawdata.dropna(subset=['eventprocessedutctime'],how='all')
    rawdata['time'] = rawdata['eventprocessedutctime'].apply(lambda x: str(x))
    rawdata['time'] = rawdata['time'].apply(lambda x: x[:19])
    rawdata['time'] = rawdata['time'].apply(lambda x: datetime.strptime(x,'%Y-%m-%dT%H:%M:%S')+timedelta(hours=8))
    rawdata['eventtime'] = rawdata['time'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')) 
    return rawdata
rms = time_normer(rms)


In [22]:
def get_grouped(rawdata,room):
    #rawdata = rawdata[rawdata['taskLocation']==room]
    rawdata = rawdata.set_index(rawdata['eventtime'].map(parser.parse))
    if rawdata.shape[0]!=0:
        grouped = rawdata.groupby(pd.TimeGrouper('60s')).size()
    return grouped

bedroom = get_grouped(rms,'Bedroom').to_frame()
bedroom = timetable_generator(bedroom,'bedroom')

## time table

In [23]:
outtable = pd.DataFrame()
outroom = outtime[outtime['outdoor']=='false']
inroom = outtime[outtime['outdoor']=='true']

outroom.index = range(outroom.shape[0])
outroom = outroom.drop(outroom.index[len(outroom)-1])


In [24]:
outroom.head()

,time,outdoor
0,2017-03-30 08:51:00,false
1,2017-03-30 13:10:00,false
2,2017-03-30 14:27:00,false
3,2017-03-30 17:50:00,false
4,2017-03-30 18:25:00,false


In [41]:
inroom = inroom.rename(index=str, columns={ 'time': 'intime'})
inroom.index = range(inroom.shape[0])
outroom = outroom.rename(index=str, columns={ 'time': 'outtime'})
outtable = pd.concat([inroom,outroom.reset_index(drop=True)],axis=1)

In [42]:
outtable.head()

,intime,outdoor,outtime,outdoor
0,2017-03-30 11:14:00,true,2017-03-30 08:51:00,false
1,2017-03-30 13:12:00,true,2017-03-30 13:10:00,false
2,2017-03-30 16:29:00,true,2017-03-30 14:27:00,false
3,2017-03-30 18:05:00,true,2017-03-30 17:50:00,false
4,2017-03-30 18:27:00,true,2017-03-30 18:25:00,false


In [53]:
result1 = pd.merge(bathroom, bedroom, how='outer', on=['time', 'time'])
result2 = pd.merge(result1, kitchen, how='outer', on=['time', 'time'])
result = pd.merge(result2, livingroom, how='outer', on=['time', 'time'])

In [54]:
bedroom.shape

(60654, 2)

In [55]:
flag1 = bedroom.ix[0]['time']
flag2 = bedroom.ix[60653]['time']
result = result[(result['time']>flag1)&(result['time']<flag2)]
flag3 = kitchen.ix[71017]['time']
result = result[result['time']<=flag3]

In [56]:
result.fillna(0, inplace=True)
result.index = range(result.shape[0])
result.head()

,bathroom,time,bedroom,kitchen,livingroom
0,0.0,2017-04-03 13:37:00,2.0,0.0,7.0
1,0.0,2017-04-03 13:38:00,1.0,0.0,6.0
2,0.0,2017-04-03 13:39:00,2.0,0.0,0.0
3,0.0,2017-04-03 13:40:00,2.0,0.0,0.0
4,0.0,2017-04-03 13:41:00,1.0,0.0,3.0


## findding the correct the situation

In [57]:
test = result[['bedroom','bathroom','kitchen','livingroom']].T
result['location'] = test.idxmax()

In [58]:
result.head()

,bathroom,time,bedroom,kitchen,livingroom,location
0,0.0,2017-04-03 13:37:00,2.0,0.0,7.0,livingroom
1,0.0,2017-04-03 13:38:00,1.0,0.0,6.0,livingroom
2,0.0,2017-04-03 13:39:00,2.0,0.0,0.0,bedroom
3,0.0,2017-04-03 13:40:00,2.0,0.0,0.0,bedroom
4,0.0,2017-04-03 13:41:00,1.0,0.0,3.0,livingroom


## add up the outdoor time

In [59]:
i=0;n=outtable.shape[0]
while i<n:
    result.loc[(result['time']>=outtable.ix[i]['outtime'])&(result['time']<=outtable.ix[i]['intime']),'location']='outdoor'
    i=i+1

In [60]:
result = result.drop_duplicates(['time'],keep='first').reset_index(drop=True)

In [61]:
result.loc[(result['bedroom']==0)&(result['bathroom']==0)&(result['livingroom']==0)&\
    (result['kitchen']==0)&(result['location']!='outdoor'),'location']=None


In [62]:
result = result.sort('time', ascending=1)
result.fillna(method='ffill', inplace=True)
result.index = range(result.shape[0])
result.head()

/Users/Susie/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,bathroom,time,bedroom,kitchen,livingroom,location
0,0.0,2017-04-03 13:37:00,2.0,0.0,7.0,livingroom
1,0.0,2017-04-03 13:38:00,1.0,0.0,6.0,livingroom
2,0.0,2017-04-03 13:39:00,2.0,0.0,0.0,bedroom
3,0.0,2017-04-03 13:40:00,2.0,0.0,0.0,bedroom
4,0.0,2017-04-03 13:41:00,1.0,0.0,3.0,livingroom


In [63]:
result.tail()

,bathroom,time,bedroom,kitchen,livingroom,location
60541,0.0,2017-05-15 14:38:00,2.0,0.0,4.0,livingroom
60542,0.0,2017-05-15 14:39:00,2.0,0.0,1.0,bedroom
60543,0.0,2017-05-15 14:40:00,2.0,0.0,6.0,livingroom
60544,0.0,2017-05-15 14:41:00,2.0,1.0,5.0,livingroom
60545,0.0,2017-05-15 14:42:00,2.0,3.0,0.0,kitchen


In [64]:

result['eventtime'] = result['time'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
result = result.set_index(result['eventtime'].map(parser.parse))
bathroom = result[result['location']=='bathroom']
bedroom = result[result['location']=='bedroom']
kitchen = result[result['location']=='kitchen']
livingroom = result[result['location']=='livingroom']
outdoor = result[result['location']=='outdoor']

grouped_bath = bathroom.groupby(pd.TimeGrouper('1800s')).size()
grouped_bed = bedroom.groupby(pd.TimeGrouper('1800s')).size()
grouped_kit = kitchen.groupby(pd.TimeGrouper('1800s')).size()
grouped_living = livingroom.groupby(pd.TimeGrouper('1800s')).size()
grouped_out = outdoor.groupby(pd.TimeGrouper('1800s')).size()

grouped_bath = timetable_generator(grouped_bath.to_frame(),'bathroom')
grouped_kit = timetable_generator(grouped_kit.to_frame(),'kitchen')
grouped_living = timetable_generator(grouped_living.to_frame(),'livingroom')
grouped_bed = timetable_generator(grouped_bed.to_frame(),'bedroom')

grouped_out = timetable_generator(grouped_out.to_frame(),'outdoor')

In [65]:
grouped_out = grouped_out.rename(index=str, columns={'bedroom': 'outdoor'})

In [66]:
result1 = pd.merge(grouped_bath, grouped_bed, how='outer', on=['time', 'time'])
result2 = pd.merge(result1, grouped_kit, how='outer', on=['time', 'time'])
result3 = pd.merge(result2, grouped_out, how='outer', on=['time', 'time'])

timetable_30min = pd.merge(result3, grouped_living, how='outer', on=['time', 'time'])

timetable_30min.fillna(0, inplace=True)
timetable_30min.head()

,bathroom,time,bedroom,kitchen,outdoor,livingroom
0,2.0,2017-04-03 17:00:00,1,0.0,27.0,0
1,1.0,2017-04-03 17:30:00,6,23.0,0.0,0
2,3.0,2017-04-03 18:00:00,1,20.0,0.0,6
3,10.0,2017-04-03 18:30:00,8,5.0,0.0,7
4,3.0,2017-04-03 19:00:00,23,4.0,0.0,0


In [67]:
test = timetable_30min[['bedroom','bathroom','kitchen','livingroom','outdoor']].T
timetable_30min['location'] = test.idxmax()
timetable_30min.head()

,bathroom,time,bedroom,kitchen,outdoor,livingroom,location
0,2.0,2017-04-03 17:00:00,1,0.0,27.0,0,outdoor
1,1.0,2017-04-03 17:30:00,6,23.0,0.0,0,kitchen
2,3.0,2017-04-03 18:00:00,1,20.0,0.0,6,kitchen
3,10.0,2017-04-03 18:30:00,8,5.0,0.0,7,bathroom
4,3.0,2017-04-03 19:00:00,23,4.0,0.0,0,bedroom


In [68]:
timetable_30min['hour'] = timetable_30min['time'].apply(lambda x: x.strftime('%H:%M:%S'))
timetable_30min['date'] = timetable_30min['time'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [69]:
timetable_30min.head()

,bathroom,time,bedroom,kitchen,outdoor,livingroom,location,hour,date
0,2.0,2017-04-03 17:00:00,1,0.0,27.0,0,outdoor,17:00:00,2017-04-03
1,1.0,2017-04-03 17:30:00,6,23.0,0.0,0,kitchen,17:30:00,2017-04-03
2,3.0,2017-04-03 18:00:00,1,20.0,0.0,6,kitchen,18:00:00,2017-04-03
3,10.0,2017-04-03 18:30:00,8,5.0,0.0,7,bathroom,18:30:00,2017-04-03
4,3.0,2017-04-03 19:00:00,23,4.0,0.0,0,bedroom,19:00:00,2017-04-03


In [70]:
timelist = timetable_30min['hour'].unique().tolist()
timelist.sort()

In [71]:
temp = []
for time in timelist:
    timepoint = timetable_30min[timetable_30min['hour']==time]
    temp.append(timepoint['location'].value_counts())

In [72]:
final = pd.DataFrame(temp)

In [73]:
final['time'] = timelist

In [74]:
final.fillna(0, inplace=True)

In [75]:
final.index = range(final.shape[0])

In [77]:
final.to_csv("/Users/Susie/Desktop/RMS_advent01_from_1st_April/pattern1.csv")